In [275]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score 

In [276]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [277]:
from tensorflow import keras
from keras.utils import to_categorical

In [278]:
pd.set_option('display.max_columns', None)

# Task 3

In [279]:
df = pd.read_csv('ODI_Match_info.csv')

In [280]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1389389,2023/24,Indore,2023/09/24,India,Australia,Australia,field,D/L,1,India,99,0,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,KN Ananthapadmanabhan
1,1336129,2023,Nottingham,2023/09/23,England,Ireland,Ireland,field,normal,0,England,48,0,WG Jacks,"Trent Bridge, Nottingham",DJ Millns,RJ Tucker,PR Reiffel
2,1395701,2023,Dhaka,2023/09/23,New Zealand,Bangladesh,New Zealand,bat,normal,0,New Zealand,86,0,IS Sodhi,"Shere Bangla National Stadium, Mirpur",M Erasmus,Sharfuddoula,Nitin Menon
3,1389388,2023/24,Chandigarh,2023/09/22,Australia,India,India,field,normal,0,India,0,5,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,J Madanagopal
4,1395700,2023,Dhaka,2023/09/21,New Zealand,Bangladesh,Bangladesh,field,normal,0,NaN,0,0,NaN,"Shere Bangla National Stadium, Mirpur",Nitin Menon,Sharfuddoula,M Erasmus


In [281]:
df3 = pd.read_csv('matches.csv')

In [282]:
df3.head()

,season,team1,team2,date,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,match_referee,winner,winner_runs,winner_wickets,match_type
0,2023/24,England,New Zealand,2023/10/05,1,Narendra Modi Stadium,Ahmedabad,New Zealand,field,R Ravindra,HDPK Dharmasena,Nitin Menon,Sharfuddoula,J Srinath,New Zealand,NaN,9.0,Group
1,2023/24,Pakistan,Netherlands,2023/10/06,2,Rajiv Gandhi International Stadium,Hyderabad,Netherlands,field,Saud Shakeel,AT Holdstock,CM Brown,PR Reiffel,JJ Crowe,Pakistan,81.0,NaN,Group
2,2023/24,Afghanistan,Bangladesh,2023/10/07,3,Himachal Pradesh Cricket Association Stadium,Dharamsala,Bangladesh,field,Mehedi Hasan Miraz,JS Wilson,HDPK Dharmasena,M Erasmus,AJ Pycroft,Bangladesh,NaN,6.0,Group
3,2023/24,South Africa,Sri Lanka,2023/10/07,4,Arun Jaitley Stadium,Delhi,Sri Lanka,field,AK Markram,RK Illingworth,Sharfuddoula,AG Wharf,J Srinath,South Africa,102.0,NaN,Group
4,2023/24,Australia,India,2023/10/08,5,MA Chidambaram Stadium,Chennai,Australia,bat,KL Rahul,CB Gaffaney,RA Kettleborough,AT Holdstock,RB Richardson,India,NaN,6.0,Group


In [283]:
df3.shape

(32, 18)

In [284]:
df.columns.to_list(), len(df.columns)

(['id',
  'season',
  'city',
  'date',
  'team1',
  'team2',
  'toss_winner',
  'toss_decision',
  'result',
  'dl_applied',
  'winner',
  'win_by_runs',
  'win_by_wickets',
  'player_of_match',
  'venue',
  'umpire1',
  'umpire2',
  'umpire3'],
 18)

In [285]:
df3.columns.to_list(), len(df3.columns)

(['season',
  'team1',
  'team2',
  'date',
  'match_number',
  'venue',
  'city',
  'toss_winner',
  'toss_decision',
  'player_of_match',
  'umpire1',
  'umpire2',
  'reserve_umpire',
  'match_referee',
  'winner',
  'winner_runs',
  'winner_wickets',
  'match_type'],
 18)

In [286]:
df3 = df3.drop(['match_type','winner_runs','winner_wickets','match_referee'], axis = 1)
df = df.drop(['dl_applied','win_by_runs','win_by_wickets','result'], axis = 1)

In [287]:
df.columns.to_list(),df3.columns.to_list(),len(df.columns),len(df3.columns)

(['id',
  'season',
  'city',
  'date',
  'team1',
  'team2',
  'toss_winner',
  'toss_decision',
  'winner',
  'player_of_match',
  'venue',
  'umpire1',
  'umpire2',
  'umpire3'],
 ['season',
  'team1',
  'team2',
  'date',
  'match_number',
  'venue',
  'city',
  'toss_winner',
  'toss_decision',
  'player_of_match',
  'umpire1',
  'umpire2',
  'reserve_umpire',
  'winner'],
 14,
 14)

In [288]:
df3.head(2)

,season,team1,team2,date,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,winner
0,2023/24,England,New Zealand,2023/10/05,1,Narendra Modi Stadium,Ahmedabad,New Zealand,field,R Ravindra,HDPK Dharmasena,Nitin Menon,Sharfuddoula,New Zealand
1,2023/24,Pakistan,Netherlands,2023/10/06,2,Rajiv Gandhi International Stadium,Hyderabad,Netherlands,field,Saud Shakeel,AT Holdstock,CM Brown,PR Reiffel,Pakistan


renaming columns in order to append dataframes

In [289]:
df3 = df3.rename(columns={"match_number":"id","reserve_umpire":"umpire3"})
df3.columns.to_list()

['season',
 'team1',
 'team2',
 'date',
 'id',
 'venue',
 'city',
 'toss_winner',
 'toss_decision',
 'player_of_match',
 'umpire1',
 'umpire2',
 'umpire3',
 'winner']

reordering columns in order to append dataframes

In [290]:
df3 = df3[df.columns]

In [291]:
df3.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2023/24,Ahmedabad,2023/10/05,England,New Zealand,New Zealand,field,New Zealand,R Ravindra,Narendra Modi Stadium,HDPK Dharmasena,Nitin Menon,Sharfuddoula
1,2,2023/24,Hyderabad,2023/10/06,Pakistan,Netherlands,Netherlands,field,Pakistan,Saud Shakeel,Rajiv Gandhi International Stadium,AT Holdstock,CM Brown,PR Reiffel
2,3,2023/24,Dharamsala,2023/10/07,Afghanistan,Bangladesh,Bangladesh,field,Bangladesh,Mehedi Hasan Miraz,Himachal Pradesh Cricket Association Stadium,JS Wilson,HDPK Dharmasena,M Erasmus
3,4,2023/24,Delhi,2023/10/07,South Africa,Sri Lanka,Sri Lanka,field,South Africa,AK Markram,Arun Jaitley Stadium,RK Illingworth,Sharfuddoula,AG Wharf
4,5,2023/24,Chennai,2023/10/08,Australia,India,Australia,bat,India,KL Rahul,MA Chidambaram Stadium,CB Gaffaney,RA Kettleborough,AT Holdstock


In [292]:
df[df['season']=="2023/24"]

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
0,1389389,2023/24,Indore,2023/09/24,India,Australia,Australia,field,India,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,KN Ananthapadmanabhan
3,1389388,2023/24,Chandigarh,2023/09/22,Australia,India,India,field,India,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,J Madanagopal
6,1373575,2023/24,Johannesburg,2023/09/17,South Africa,Australia,Australia,field,South Africa,M Jansen,"The Wanderers Stadium, Johannesburg",A Paleker,RA Kettleborough,Nitin Menon
9,1373574,2023/24,Centurion,2023/09/15,South Africa,Australia,Australia,field,South Africa,H Klaasen,"SuperSport Park, Centurion",A Paleker,Nitin Menon,RA Kettleborough
12,1373573,2023/24,Potchefstroom,2023/09/12,South Africa,Australia,Australia,field,South Africa,AK Markram,"Sedgars Park, Potchefstroom",BP Jele,RA Kettleborough,Nitin Menon
16,1373572,2023/24,Bloemfontein,2023/09/09,Australia,South Africa,South Africa,field,Australia,M Labuschagne,"Mangaung Oval, Bloemfontein",BP Jele,Nitin Menon,RA Kettleborough
19,1373571,2023/24,Bloemfontein,2023/09/07,South Africa,Australia,Australia,field,Australia,M Labuschagne,"Mangaung Oval, Bloemfontein",A Paleker,RA Kettleborough,Nitin Menon


In [293]:
df3[df3['season']=="2023/24"]

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2023/24,Ahmedabad,2023/10/05,England,New Zealand,New Zealand,field,New Zealand,R Ravindra,Narendra Modi Stadium,HDPK Dharmasena,Nitin Menon,Sharfuddoula
1,2,2023/24,Hyderabad,2023/10/06,Pakistan,Netherlands,Netherlands,field,Pakistan,Saud Shakeel,Rajiv Gandhi International Stadium,AT Holdstock,CM Brown,PR Reiffel
2,3,2023/24,Dharamsala,2023/10/07,Afghanistan,Bangladesh,Bangladesh,field,Bangladesh,Mehedi Hasan Miraz,Himachal Pradesh Cricket Association Stadium,JS Wilson,HDPK Dharmasena,M Erasmus
3,4,2023/24,Delhi,2023/10/07,South Africa,Sri Lanka,Sri Lanka,field,South Africa,AK Markram,Arun Jaitley Stadium,RK Illingworth,Sharfuddoula,AG Wharf
4,5,2023/24,Chennai,2023/10/08,Australia,India,Australia,bat,India,KL Rahul,MA Chidambaram Stadium,CB Gaffaney,RA Kettleborough,AT Holdstock
5,6,2023/24,Hyderabad,2023/10/09,New Zealand,Netherlands,Netherlands,field,New Zealand,MJ Santner,Rajiv Gandhi International Stadium,PR Reiffel,RJ Tucker,Nitin Menon
6,7,2023/24,Dharamsala,2023/10/10,England,Bangladesh,Bangladesh,field,England,DJ Malan,Himachal Pradesh Cricket Association Stadium,Ahsan Raza,P Wilson,HDPK Dharmasena
7,8,2023/24,Hyderabad,2023/10/10,Sri Lanka,Pakistan,Sri Lanka,bat,Pakistan,Mohammad Rizwan,Rajiv Gandhi International Stadium,AG Wharf,CB Gaffaney,RJ Tucker
8,9,2023/24,Delhi,2023/10/11,Afghanistan,India,Afghanistan,bat,India,RG Sharma,Arun Jaitley Stadium,MA Gough,PR Reiffel,Sharfuddoula
9,10,2023/24,Lucknow,2023/10/12,South Africa,Australia,Australia,field,South Africa,Q de Kock,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,JS Wilson,RK Illingworth,CM Brown


In [294]:
df3['id']=df['id'].max()+1

**Concatenating present world cup data and previous odi data**

In [295]:
df = pd.concat([df, df3], sort=False)

## Handling Null Values and dropping columns

In [296]:
df.isna().sum()

id                   0
season               0
city               310
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner             120
player_of_match    151
venue                0
umpire1              0
umpire2              0
umpire3            283
dtype: int64

**removing matches where winner not concluded**

In [297]:
df = df.dropna(axis = 0,subset=['winner'])

In [298]:
df[df['player_of_match'].isna()]

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,umpire3
165,1341974,2022/23,Windhoek,2022/11/26,United States of America,Namibia,Namibia,field,Namibia,NaN,"Wanderers Cricket Ground, Windhoek",A Louw,BP Jele,NaN
183,1331369,2022,Port Moresby,2022/09/20,Namibia,Papua New Guinea,Namibia,bat,Namibia,NaN,"Amini Park, Port Moresby",DM Koch,L Oala,NaN
185,1331367,2022,Port Moresby,2022/09/15,Namibia,United States of America,Namibia,bat,Namibia,NaN,"Amini Park, Port Moresby",A Kapa,L Oala,NaN
186,1331366,2022,Port Moresby,2022/09/13,Papua New Guinea,United States of America,United States of America,field,Papua New Guinea,NaN,"Amini Park, Port Moresby",A Kapa,DM Koch,NaN
254,1312802,2022,Pearland,2022/06/03,Scotland,United Arab Emirates,Scotland,bat,United Arab Emirates,NaN,"Moosa Cricket Stadium, Pearland",SN Bandekar,Vijaya Mallela,NaN
266,1308244,2022,Dubai,2022/04/09,Scotland,Papua New Guinea,Papua New Guinea,field,Scotland,NaN,Dubai International Cricket Stadium,Akbar Ali,BB Pradhan,NaN
297,1299587,2021/22,Al Amarat,2022/02/06,Oman,United Arab Emirates,United Arab Emirates,field,United Arab Emirates,NaN,Al Amerat Cricket Ground Oman Cricket (Ministr...,AT Holdstock,Akbar Ali,NaN
360,1260085,2021,Rotterdam,2021/05/20,Netherlands,Scotland,Netherlands,bat,Scotland,NaN,"Hazelaarweg, Rotterdam",Adriaan van den Dries,N Bathi,NaN
361,1260084,2021,Rotterdam,2021/05/19,Netherlands,Scotland,Netherlands,bat,Netherlands,NaN,"Hazelaarweg, Rotterdam",N Bathi,Rizwan Akram,Adriaan van den Dries
447,1196672,2019,Aberdeen,2019/08/17,Scotland,Papua New Guinea,Scotland,field,Scotland,NaN,Mannofield Park,DA Haggo,R Black,NaN


Removing all columns containing only null values

In [299]:
df = df.dropna(how='all')

In [300]:
df.isna().sum()

id                   0
season               0
city               297
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
winner               0
player_of_match     57
venue                0
umpire1              0
umpire2              0
umpire3            268
dtype: int64

In [301]:
df = df.drop(['umpire3'], axis = 1)

In [302]:
df[df['city'].isna()]

,id,season,city,date,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2
383,1223956,2020/21,NaN,2020/11/29,Australia,India,Australia,bat,Australia,SPD Smith,Sydney Cricket Ground,GA Abood,PR Reiffel
384,1223955,2020/21,NaN,2020/11/27,Australia,India,Australia,bat,Australia,SPD Smith,Sydney Cricket Ground,RJ Tucker,SJ Nogajski
386,1233462,2020/21,NaN,2020/11/01,Zimbabwe,Pakistan,Zimbabwe,bat,Pakistan,Iftikhar Ahmed,Rawalpindi Cricket Stadium,Asif Yaqoob,Aleem Dar
387,1233461,2020/21,NaN,2020/10/30,Pakistan,Zimbabwe,Pakistan,bat,Pakistan,BRM Taylor,Rawalpindi Cricket Stadium,Aleem Dar,Ahsan Raza
579,1144987,2018/19,NaN,2018/11/09,Australia,South Africa,South Africa,field,Australia,AJ Finch,Adelaide Oval,MA Gough,GA Abood
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,217481,2005,NaN,2005/08/29,India,Zimbabwe,Zimbabwe,field,India,IK Pathan,Harare Sports Club,IL Howell,KC Barbour
2140,214852,2005,NaN,2005/08/02,Sri Lanka,West Indies,Sri Lanka,bat,Sri Lanka,MF Maharoof,Rangiri Dambulla International Stadium,EAR de Silva,SJA Taufel
2141,214742,2005,NaN,2005/07/31,West Indies,India,West Indies,bat,India,R Dravid,Rangiri Dambulla International Stadium,DJ Harper,TH Wijewardene
2168,65666,2004/05,NaN,2005/02/04,Australia,Pakistan,Australia,bat,Australia,A Symonds,Melbourne Cricket Ground,BF Bowden,SJ Davis


filling in null city column values using venue column

In [303]:
# First, create a dictionary mapping venues to their corresponding cities
venue_city_mapping = df.dropna(subset=['city']).set_index('venue')['city'].to_dict()

# Function to fill missing city values based on venue
def fill_missing_city(row):
    if pd.isnull(row['city']):
        return venue_city_mapping.get(row['venue'], row['city'])
    return row['city']

# Apply the function to fill missing city values
df['city'] = df.apply(fill_missing_city, axis=1)


In [304]:
 df[df['city'].isna()], df[df['venue']== 'Perth Stadium']


(           id   season city        date        team1         team2  \
 581   1144986  2018/19  NaN  2018/11/04    Australia  South Africa   
 675   1072314  2017/18  NaN  2018/01/28      England     Australia   
 739   1104479     2017  NaN  2017/07/02    Sri Lanka      Zimbabwe   
 741   1104478     2017  NaN  2017/06/30    Sri Lanka      Zimbabwe   
 1170   661689  2013/14  NaN  2014/01/01  New Zealand   West Indies   
 1362   543882  2011/12  NaN  2012/02/20        Kenya       Ireland   
 1365   543881  2011/12  NaN  2012/02/18        Kenya       Ireland   
 1701   392617     2009  NaN  2009/04/24    Australia      Pakistan   
 1702   392616     2009  NaN  2009/04/22    Australia      Pakistan   
 1814   323949  2007/08  NaN  2008/03/09   Bangladesh  South Africa   
 1834   325804  2007/08  NaN  2008/02/02     Pakistan      Zimbabwe   
 1843   300428  2007/08  NaN  2007/12/31  New Zealand    Bangladesh   
 2068   238173  2005/06  NaN  2006/04/23   Bangladesh     Australia   
 2087 

filling null city values left out by above method

In [305]:
df.loc[df['venue'] == 'Perth Stadium', 'city'] = 'Australia'
df.loc[df['venue'] == 'Galle International Stadium', 'city'] = 'Sri Lanka'
df.loc[df['venue'] == 'Queenstown Events Centre', 'city'] = 'New Zealand'
df.loc[df['venue'] == 'Mombasa Sports Club Ground', 'city'] = 'Kenya'
df.loc[df['venue'] == 'Dubai Sports City Cricket Stadium', 'city'] = 'United Arab Emirates'
df.loc[df['venue'] == 'Chittagong Divisional Stadium', 'city'] = 'Bangladesh'
df.loc[df['venue'] == 'Sheikhupura Stadium', 'city'] = 'Pakistan'

In [306]:
df.isna().sum()

id                  0
season              0
city                0
date                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    57
venue               0
umpire1             0
umpire2             0
dtype: int64

In [307]:
df['winner'].value_counts()

India                       296
Australia                   279
South Africa                218
Sri Lanka                   216
England                     210
Pakistan                    202
New Zealand                 188
West Indies                 136
Bangladesh                  129
Zimbabwe                     67
Afghanistan                  59
Ireland                      53
Scotland                     45
United Arab Emirates         34
Nepal                        28
Oman                         22
United States of America     22
Namibia                      21
Netherlands                  19
Kenya                        13
Papua New Guinea             11
Canada                       10
Hong Kong                     6
Asia XI                       3
Bermuda                       2
Jersey                        1
Africa XI                     1
Name: winner, dtype: int64

converting date to dayofyear

In [308]:
df['day_of_year'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# Extract the day of the year and create a new column
df['day_of_year'] = df['day_of_year'].dt.dayofyear

In [309]:
df.isna().sum()

id                  0
season              0
city                0
date                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    57
venue               0
umpire1             0
umpire2             0
day_of_year         0
dtype: int64

In [310]:
df = df.drop(['date'], axis = 1)

In [311]:
df.head()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,day_of_year
0,1389389,2023/24,Indore,India,Australia,Australia,field,India,SS Iyer,"Holkar Cricket Stadium, Indore",J Madanagopal,HDPK Dharmasena,267
1,1336129,2023,Nottingham,England,Ireland,Ireland,field,England,WG Jacks,"Trent Bridge, Nottingham",DJ Millns,RJ Tucker,266
2,1395701,2023,Dhaka,New Zealand,Bangladesh,New Zealand,bat,New Zealand,IS Sodhi,"Shere Bangla National Stadium, Mirpur",M Erasmus,Sharfuddoula,266
3,1389388,2023/24,Chandigarh,Australia,India,India,field,India,Mohammed Shami,"Punjab Cricket Association IS Bindra Stadium, ...",KN Ananthapadmanabhan,HDPK Dharmasena,265
5,1388414,2023,Colombo,Sri Lanka,India,Sri Lanka,bat,India,Mohammed Siraj,"R Premadasa Stadium, Colombo",Ahmed Shah Pakteen,RK Illingworth,260


In [312]:
# venue has both values 'Wankhede Stadium' and 'Wankhede Stadium, Mumbai' , removing location from venue

df['venue'] = df['venue'].str.split(',').str.get(0)

In [313]:
new_df = df[['season', 'city', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'venue', 'umpire1', 
       'day_of_year']]

removing some teams with less number of matches won

In [314]:
values_to_drop = ['Scotland',  
'United Arab Emirates',      
'Nepal',                        
'Oman',                         
'United States of America',    
'Namibia',                    
'Netherlands',                  
'Kenya',                        
'Papua New Guinea',             
'Canada',                       
'Hong Kong',                    
'Asia XI',                    
'Bermuda',                    
'Jersey',                        
'Africa XI',
'ICC World XI' ]

# Drop rows where the 'winner' column has values in 'values_to_drop'
new_df = df[~df['winner'].isin(values_to_drop)]
new_df = new_df[~new_df['team1'].isin(values_to_drop)]
new_df = new_df[~new_df['team2'].isin(values_to_drop)]

In [315]:
new_df['team2'].value_counts(), new_df.shape

(Sri Lanka       249
 Pakistan        234
 India           223
 Australia       180
 West Indies     175
 South Africa    171
 England         169
 New Zealand     154
 Zimbabwe        124
 Bangladesh      111
 Afghanistan      51
 Ireland          50
 Name: team2, dtype: int64,
 (1891, 13))

## Encoding Data

In [316]:
TeamMapping ={'India':1, 'Australia':2,'South Africa':3,'Sri Lanka':4,'England':5,'Pakistan':6,'New Zealand':7,     
'West Indies':8,'Bangladesh':9,'Zimbabwe':10,'Afghanistan':11,'Ireland':12}
new_df['team1'] = new_df['team1'].map(TeamMapping)

new_df['team2'] = new_df['team2'].map(TeamMapping)

new_df['winner'] = new_df['winner'].map(TeamMapping)

new_df['toss_winner'] = new_df['toss_winner'].map(TeamMapping)

In [317]:
new_df.isna().sum()

id                  0
season              0
city                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    18
venue               0
umpire1             0
umpire2             0
day_of_year         0
dtype: int64

In [318]:
#creating a copyof dataframe
copy_df = new_df.copy()

In [319]:
le = LabelEncoder()

new_df['season'] = le.fit_transform(new_df['season'])
# Create a dictionary mapping original values to encoded values
mapping_season = dict(zip(copy_df['season'], new_df['season']))

new_df['toss_decision'] = le.fit_transform(new_df['toss_decision'])
# Create a dictionary mapping original values to encoded values
mapping_toss_decision = dict(zip(copy_df['toss_decision'], new_df['toss_decision']))

new_df['city'] = le.fit_transform(new_df['city'])
# Create a dictionary mapping original values to encoded values
mapping_city = dict(zip(copy_df['city'], new_df['city']))

new_df['venue'] = le.fit_transform(new_df['venue'])
# Create a dictionary mapping original values to encoded values
mapping_venue = dict(zip(copy_df['venue'], new_df['venue']))

new_df['umpire1'] = le.fit_transform(new_df['umpire1'])
# Create a dictionary mapping original values to encoded values
mapping_umpire = dict(zip(copy_df['umpire1'], new_df['umpire1']))

In [320]:
new_df.tail()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,day_of_year
26,1395702,42,41,2,7,7,1,2,TM Head,55,62,Sharfuddoula,301
28,1395702,42,82,1,5,5,1,1,RG Sharma,14,7,RJ Tucker,302
29,1395702,42,105,4,11,11,1,11,Fazalhaq Farooqi,76,25,RK Illingworth,303
30,1395702,42,77,9,6,9,0,6,Fakhar Zaman,41,76,RA Kettleborough,304
31,1395702,42,105,3,7,7,1,3,HE van der Dussen,76,12,HDPK Dharmasena,305


In [321]:
new_df.shape

(1891, 13)

## Training model to predict winner of match

In [322]:
X = new_df[['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision',
        'venue', 'umpire1', 'day_of_year']]
y = new_df['winner']

## knn

In [323]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [324]:
neigh = KNeighborsClassifier(n_neighbors=6)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

In [325]:
y_pred = neigh.predict(X_test)

In [326]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.2638522427440633
Precision:0.26336782107401824
Recall:0.2638522427440633
f1 score:0.2590502752734971


In [327]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(neigh, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

Accuracy score of each fold:  [0.18469657 0.21428571 0.20899471 0.26190476 0.2010582 ]
Mean accuracy score:  0.2141879912328461


## naive bayes

In [328]:
bayes = CategoricalNB()
bayes.fit(X_train, y_train)


CategoricalNB()

In [329]:
y_pred = bayes.predict(X_test)

In [330]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.5725593667546174
Precision:0.5948567934868707
Recall:0.5725593667546174
f1 score:0.5761990461266114


## random forest

In [331]:
forest = RandomForestClassifier(max_depth=10, random_state=0)
forest.fit(X_train, y_train)


y_pred = forest.predict(X_test)

In [332]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.6200527704485488
Precision:0.6382601914871178
Recall:0.6200527704485488
f1 score:0.6077403447367002


In [333]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(forest, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

Accuracy score of each fold:  [0.65435356 0.57671958 0.61375661 0.65873016 0.61375661]
Mean accuracy score:  0.623463304993648


## Using Neural Network

In [334]:
mlp = MLPClassifier(hidden_layer_sizes=(22,100,100,22),random_state=1, max_iter=300).fit(X_train, y_train)
y_pred = mlp.predict(X_test)

C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [335]:
y_pred2 = mlp.predict(X_train)
accuracy_score(y_train, y_pred2)

0.5866402116402116

In [336]:
print(f"Accuracy:{accuracy_score(y_test, y_pred)}")
print(f"Precision:{precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall:{recall_score(y_test, y_pred, average='weighted')}")
print(f"f1 score:{f1_score(y_test, y_pred, average='weighted')}")

Accuracy:0.2532981530343008
Precision:0.2910656920415019
Recall:0.2532981530343008
f1 score:0.25766107336542987


In [337]:
k_fold = KFold(n_splits = 5, random_state = None) 
score = cross_val_score(mlp, X, y, cv = k_fold)  
  
# Calculating mean accuracy score  
mean_acc_score = sum(score) / len(score)  
   
print("Accuracy score of each fold: ", score)  
print("Mean accuracy score: ", mean_acc_score)  

C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\dhair\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy score of each fold:  [0.17941953 0.23280423 0.26719577 0.23015873 0.1957672 ]
Mean accuracy score:  0.22106909019837778


In [338]:
X.columns

Index(['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'venue', 'umpire1', 'day_of_year'],
      dtype='object')

In [339]:
umpire_list = ['HDPK Dharmasena', 'M Erasmus', 'CB Gaffaney', 'MA Gough', 'AT Holdstock', 'RK Illingworth', 'RA Kettleborough', 'Nitin Menon', 'Ahsan Raza', 'PR Reiffel', 'Sharfuddoula', 'RJ Tucker', 'AG Wharf', 'JS Wilson', 'CM Brown', 'P Wilson']

In [340]:
sem_fin1_data = [[mapping_season['2023/24'],mapping_city['Mumbai'],TeamMapping['India'],TeamMapping['New Zealand'],'','',mapping_venue['Wankhede Stadium'],'',319]]
sem_fin2_data = [[mapping_season['2023/24'],mapping_city['Kolkata'],TeamMapping['South Africa'],TeamMapping['Australia'],'','',mapping_venue['Eden Gardens'],'',320]]
fin_data = [[mapping_season['2023/24'],mapping_city['Ahmedabad'],'','','','',mapping_venue['Narendra Modi Stadium'],'',323]]

sem_fin1_df = pd.DataFrame(sem_fin1_data,columns = X.columns)
sem_fin2_df = pd.DataFrame(sem_fin2_data,columns = X.columns)
fin_df =  pd.DataFrame(fin_data,columns = X.columns)
score = pd.DataFrame([[0,0,0,0]],columns = ['India','New Zealand','South Africa','Australia'])


In [341]:
sem_fin1_data

[[42, 89, 1, 7, '', '', 156, '', 319]]

In [342]:
sem_fin2_data

[[42, 77, 3, 2, '', '', 41, '', 320]]

### Creating dataframe for semi final match , for columns whose values are not available, filling in random values and doing this mutiple times and calculating result

### First winners of semi final matches are predicted and the results are plugged in dataframe of final match, prediction on this dataframe is the result, this result is calculated for multiple different situations and the team predicted majority of times as winner, is declared final winner 

In [343]:
import random
for i in range(100):
    
    #semi final 1
    if random.randrange(2):
        sem_fin1_df['toss_winner'] = sem_fin1_df['team1']
    else:
        sem_fin1_df['toss_winner'] = sem_fin1_df['team2']
        
    sem_fin1_df['toss_decision'] = random.randrange(2)
    sem_fin1_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #semi final 2
    if random.randrange(2):
        sem_fin2_df['toss_winner'] = sem_fin2_df['team1']
    else:
        sem_fin2_df['toss_winner'] = sem_fin2_df['team2']
        
    sem_fin2_df['toss_decision'] = random.randrange(2)
    sem_fin2_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    sem_fin_pred1 = forest.predict(sem_fin1_df)
    sem_fin_pred2 = forest.predict(sem_fin2_df)
    
    #finals
    fin_df['team1'] =  sem_fin_pred1
    fin_df['team2'] =  sem_fin_pred2
    
    if random.randrange(2):
        fin_df['toss_winner'] = fin_df['team1']
    else:
        fin_df['toss_winner'] = fin_df['team2']
        
    fin_df['toss_decision'] = random.randrange(2)
    fin_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #finals result
    fin_pred = forest.predict(fin_df)
    
    if fin_pred == 1 : score['India']+=1
    elif fin_pred == 3  : score['South Africa']+=1
    elif fin_pred == 7  : score['New Zealand']+=1
    else : score['Australia']+=1    
    
print("Chances of winning out of 100, according to random forest model")   
print(score)

Chances of winning out of 100, according to random forest model
   India  New Zealand  South Africa  Australia
0     94            1             5          0


In [344]:
print("Chances of winning out of 100, according to random forest model")   
print(score)

score = dict.fromkeys(score, 0)

Chances of winning out of 100, according to random forest model
   India  New Zealand  South Africa  Australia
0     94            1             5          0


### using neural network model

In [345]:
import random
for i in range(100):
    
    #semi final 1
    if random.randrange(2):
        sem_fin1_df['toss_winner'] = sem_fin1_df['team1']
    else:
        sem_fin1_df['toss_winner'] = sem_fin1_df['team2']
        
    sem_fin1_df['toss_decision'] = random.randrange(2)
    sem_fin1_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #semi final 2
    if random.randrange(2):
        sem_fin2_df['toss_winner'] = sem_fin2_df['team1']
    else:
        sem_fin2_df['toss_winner'] = sem_fin2_df['team2']
        
    sem_fin2_df['toss_decision'] = random.randrange(2)
    sem_fin2_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    sem_fin_pred1 = mlp.predict(sem_fin1_df)
    sem_fin_pred2 = mlp.predict(sem_fin2_df)
    
    #finals
    fin_df['team1'] =  sem_fin_pred1
    fin_df['team2'] =  sem_fin_pred2
    
    if random.randrange(2):
        fin_df['toss_winner'] = fin_df['team1']
    else:
        fin_df['toss_winner'] = fin_df['team2']
        
    fin_df['toss_decision'] = random.randrange(2)
    fin_df['umpire1'] = mapping_umpire[umpire_list[random.randrange(16)]]
    
    #finals result
    fin_pred = mlp.predict(fin_df)
    
    if fin_pred == 1 : score['India']+=1
    elif fin_pred == 3  : score['South Africa']+=1
    elif fin_pred == 7  : score['New Zealand']+=1
    else : score['Australia']+=1    
    
print("Chances of winning out of 100, according to neural network model")   
print(score)

Chances of winning out of 100, according to neural network model
{'India': 87, 'New Zealand': 0, 'South Africa': 0, 'Australia': 13}


In [346]:
print("Chances of winning out of 100, according to neural network model")   
print(score)

score = dict.fromkeys(score, 0)

Chances of winning out of 100, according to neural network model
{'India': 87, 'New Zealand': 0, 'South Africa': 0, 'Australia': 13}


## Task1: 
### Predicting player of match

In [347]:
new_df.head()

,id,season,city,team1,team2,toss_winner,toss_decision,winner,player_of_match,venue,umpire1,umpire2,day_of_year
0,1389389,42,64,1,2,2,1,1,SS Iyer,56,50,HDPK Dharmasena,267
1,1336129,41,96,5,12,12,1,5,WG Jacks,149,31,RJ Tucker,266
2,1395701,41,40,7,9,7,0,7,IS Sodhi,133,62,Sharfuddoula,266
3,1389388,42,27,2,1,1,1,1,Mohammed Shami,100,57,HDPK Dharmasena,265
5,1388414,41,34,4,1,4,0,1,Mohammed Siraj,106,10,RK Illingworth,260


In [348]:
new_df.isna().sum()

id                  0
season              0
city                0
team1               0
team2               0
toss_winner         0
toss_decision       0
winner              0
player_of_match    18
venue               0
umpire1             0
umpire2             0
day_of_year         0
dtype: int64

In [349]:
new_df = new_df.dropna(axis = 0,subset=['player_of_match'])
copy_df = copy_df.dropna(axis = 0,subset=['player_of_match'])

In [350]:
new_df = new_df.drop(['umpire2','id'], axis=1)

In [351]:
new_df['player_of_match'] = le.fit_transform(new_df['player_of_match'])
# Create a dictionary mapping original values to encoded values
mapping_player = dict(zip(copy_df['player_of_match'], new_df['player_of_match']))

In [352]:
len(mapping_player)

493

In [353]:
X = new_df[['season', 'city', 'team1', 'team2', 'toss_winner', 'toss_decision','winner',
        'venue', 'umpire1', 'day_of_year']]
y = new_df['player_of_match']


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=42)

In [354]:
np_Xtrain = Xtrain.to_numpy(dtype='float32')
np_ytrain = ytrain.to_numpy(dtype='float32')
np_Xtest = Xtest.to_numpy(dtype='float32')
np_ytest = ytest.to_numpy(dtype='float32')

In [355]:
np_Xtrain, np_ytrain

(array([[ 33.,  31.,   3., ..., 113.,  92., 179.],
        [ 14.,  85.,   2., ...,  83.,  17.,  38.],
        [ 19.,  59.,   4., ...,  77.,  19., 206.],
        ...,
        [ 21.,  20.,  10., ..., 104., 110., 213.],
        [  9.,  86.,   9., ..., 133.,  36., 130.],
        [ 16.,  56.,   1., ...,  90.,  73., 332.]], dtype=float32),
 array([ 92., 414., 327., ..., 286., 293., 468.], dtype=float32))

In [356]:
np_y = np.array([])
np_y = np.append(np_ytrain,np_ytest,axis=0)
np_y = to_categorical(np_y)


In [357]:
np_ytrain = np_y[:1498,]
np_ytest = np_y[1498:,]

In [358]:
np_ytrain.shape, np_ytest.shape

((1498, 493), (375, 493))

In [359]:
mean = np.mean(np_Xtrain, axis=0)
np_Xtrain -= mean
np_Xtest -= mean
std = np.std(np_Xtrain, axis=0)
np_Xtrain /= std
np_Xtest /= std

In [360]:
np_Xtrain.shape[-1], np_ytrain.shape[-1]

(10, 493)

In [361]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            128, activation="relu", input_shape=(np_Xtrain.shape[-1],)
        ),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(493, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               1408      
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 493)               63597     
                                                                 
Total params: 98,029
Trainable params: 98,029
Non-trainable params: 0
_________________________________________________________________


In [362]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="categorical_crossentropy", metrics=metrics
)

In [363]:
np_Xtrain.shape,np_ytrain.shape,np_Xtest.shape, np_ytest.shape

((1498, 10), (1498, 493), (375, 10), (375, 493))

In [364]:
model.fit(
    np_Xtrain,
    np_ytrain,
    epochs=80,
    verbose=2,
    validation_data=(np_Xtest, np_ytest),
)

Epoch 1/80
47/47 - 2s - loss: 5.9545 - fn: 1498.0000 - fp: 0.0000e+00 - tn: 737016.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.6990 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 2s/epoch - 51ms/step
Epoch 2/80
47/47 - 0s - loss: 5.1861 - fn: 1498.0000 - fp: 0.0000e+00 - tn: 737016.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.6102 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 280ms/epoch - 6ms/step
Epoch 3/80
47/47 - 0s - loss: 4.5939 - fn: 1498.0000 - fp: 1.0000 - tn: 737015.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 5.4231 - val_fn: 375.0000 - val_fp: 0.0000e+00 - val_tn: 184500.0000 - val_tp: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 298ms/epoch - 6ms/step
Epoch 4/80
47/47

Epoch 28/80
47/47 - 0s - loss: 1.0471 - fn: 602.0000 - fp: 290.0000 - tn: 736726.0000 - tp: 896.0000 - precision: 0.7555 - recall: 0.5981 - val_loss: 16.0797 - val_fn: 358.0000 - val_fp: 230.0000 - val_tn: 184270.0000 - val_tp: 17.0000 - val_precision: 0.0688 - val_recall: 0.0453 - 280ms/epoch - 6ms/step
Epoch 29/80
47/47 - 0s - loss: 0.9159 - fn: 540.0000 - fp: 254.0000 - tn: 736762.0000 - tp: 958.0000 - precision: 0.7904 - recall: 0.6395 - val_loss: 16.5473 - val_fn: 355.0000 - val_fp: 226.0000 - val_tn: 184274.0000 - val_tp: 20.0000 - val_precision: 0.0813 - val_recall: 0.0533 - 282ms/epoch - 6ms/step
Epoch 30/80
47/47 - 0s - loss: 0.8844 - fn: 518.0000 - fp: 272.0000 - tn: 736744.0000 - tp: 980.0000 - precision: 0.7827 - recall: 0.6542 - val_loss: 17.0779 - val_fn: 360.0000 - val_fp: 236.0000 - val_tn: 184264.0000 - val_tp: 15.0000 - val_precision: 0.0598 - val_recall: 0.0400 - 278ms/epoch - 6ms/step
Epoch 31/80
47/47 - 0s - loss: 0.9027 - fn: 503.0000 - fp: 257.0000 - tn: 736759.0

Epoch 55/80
47/47 - 0s - loss: 0.5511 - fn: 278.0000 - fp: 189.0000 - tn: 736827.0000 - tp: 1220.0000 - precision: 0.8659 - recall: 0.8144 - val_loss: 23.0416 - val_fn: 356.0000 - val_fp: 294.0000 - val_tn: 184206.0000 - val_tp: 19.0000 - val_precision: 0.0607 - val_recall: 0.0507 - 276ms/epoch - 6ms/step
Epoch 56/80
47/47 - 0s - loss: 0.5228 - fn: 285.0000 - fp: 213.0000 - tn: 736803.0000 - tp: 1213.0000 - precision: 0.8506 - recall: 0.8097 - val_loss: 23.7189 - val_fn: 355.0000 - val_fp: 294.0000 - val_tn: 184206.0000 - val_tp: 20.0000 - val_precision: 0.0637 - val_recall: 0.0533 - 276ms/epoch - 6ms/step
Epoch 57/80
47/47 - 0s - loss: 0.6450 - fn: 319.0000 - fp: 225.0000 - tn: 736791.0000 - tp: 1179.0000 - precision: 0.8397 - recall: 0.7870 - val_loss: 23.5610 - val_fn: 359.0000 - val_fp: 297.0000 - val_tn: 184203.0000 - val_tp: 16.0000 - val_precision: 0.0511 - val_recall: 0.0427 - 327ms/epoch - 7ms/step
Epoch 58/80
47/47 - 0s - loss: 0.7506 - fn: 343.0000 - fp: 244.0000 - tn: 73677

In [365]:
model.evaluate(np_Xtest,np_ytest)

12/12 [==============================] - 0s 4ms/step - loss: 27.7323 - fn: 355.0000 - fp: 313.0000 - tn: 184187.0000 - tp: 20.0000 - precision: 0.0601 - recall: 0.0533


[27.7323055267334,
 355.0,
 313.0,
 184187.0,
 20.0,
 0.060060061514377594,
 0.0533333346247673]